# Verify ID

`ID`の重複があるかを確認する処理の試験実装。

In [34]:
import glob, json
import pandas as pd

jsonFileList = glob.glob('reports-data/*.json')

df = pd.DataFrame()
for file in jsonFileList:
	with open(file, "r", encoding='utf-8') as f:
		data = json.load(f)
		each_df = pd.DataFrame(data)
		df = pd.concat([df, each_df])

In [35]:
start_with_id_columns = df.columns.delete(15).to_list()
start_with_id_columns.insert(0, "id")
df = df.reindex(columns=start_with_id_columns)
df = df.reset_index(drop=True)

In [36]:
duplicated_df = df[df["id"].duplicated()].iloc[:, [0, 2, 3]]
if len(duplicated_df) > 0:
	print("[Error] idが重複したデータがあります。修正してください。")
	print(duplicated_df)

# pre_existing_conditions

改行文字を含む`pre_existing_conditions`を処理する際の試験実装。

In [37]:
pec_series = df['pre_existing_conditions'].str.replace("\r\n", "\n")
include_line_pec_df = df[pec_series.str.split("\n").map(lambda x: len(x) > 1)]

for index in include_line_pec_df.index:
	show_section = False
	
	pec_item = include_line_pec_df.at[index, 'pre_existing_conditions']

	pec_split_list = pec_item.split("\r\n")
	if len(pec_split_list) > 1:
		print(f"## {include_line_pec_df.at[index, 'source']['url'].split('/').pop().replace(".pdf", ".json")}, id: {include_line_pec_df.at[index, 'id']}")
		#print(f"・ {pec_split_list}")
	
	'''
	for pec in pec_item.split("\n"):
		# 併用薬のエントリのみ表示する処理
		if pec.startswith("併用薬"):
			if not show_section:
				show_section = True
				print(f"## {include_line_pec_df.at[index, 'source']['url'].split('/').pop().replace(".pdf", ".json")}, id: {include_line_pec_df.at[index, 'id']}")
			print(f"・ {pec}")
	'''
	
	if show_section:
		print()